In [1]:
import multiprocessing.pool
import functools
import requests
import pandas as pd
from dateutil.relativedelta import relativedelta
import warnings
import os
import shutil
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.dates as mdates
import matplotlib.ticker as mtick
import numpy as np
import time
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso
import ag_email_utils

In [16]:
from datetime import date

In [19]:
len(pd.date_range(start=(date.today()-relativedelta(months=backcast_months)), end=forecast_end_date, freq='MS'))

23

In [18]:
len(pd.date_range(start=pd.to_datetime((datetime.now()-relativedelta(months=backcast_months)).strftime('%Y-%m-01')), end=forecast_end_date, freq='MS'))

24

In [2]:
start_time = time.time()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('expand_frame_repr', False)
pd.options.display.float_format = '{:,.2f}'.format

############### SPECIFY (note: JODI runs can be amended on line 2613)##############################################
backcast_months = 4 # forecast back dates 4 months from today
graph_historic_years = 5 # graph data from 5 yrs back
forecast_end_date = pd.to_datetime((datetime.now()+relativedelta(years=1)).strftime('%Y-12-01')) # forecast till end of next year.
months_ahead = len(pd.date_range(start=pd.to_datetime((datetime.now()-relativedelta(months=backcast_months)).strftime('%Y-%m-01')), end=forecast_end_date, freq='MS'))
correlation_months_spec = 25 # months used for correlation
timeout_sec= 30 #time (seconds) allowed for query to run before error message
long_timeout_sec=120 #time (seconds) allowed for query to run before error message
Pdate_days_before=0 #0 is today and 1 is yesterday
correlation_utilisation_upper_limit = 1.5 # upper limit to discard utilisation rates as anomaly

IIR_Countries = ['Austria','Belgium','Denmark','Finland','France','Germany','Greece','Ireland','Italy','Netherlands','Norway','Portugal','Spain','Sweden','United Kingdom','Turkey','Czech Republic','Hungary','Poland','Slovakia','Switzerland']
NWE_Countries = ['Belgium', 'Denmark', 'Finland', 'Germany', 'Ireland', 'Netherlands', 'Norway', 'Sweden', 'United Kingdom','France NWE']
Med_Countries = ['Austria', 'Czech Republic', 'Greece', 'Hungary', 'Italy', 'Poland', 'Portugal', 'Slovakia', 'Spain','Switzerland', 'Turkey', 'France Med']
Med_Countries_for_table = ['Austria', 'Greece', 'Italy', 'Portugal', 'Spain','Switzerland', 'Turkey', 'France Med']

#Make sure IEA and JODI country list mataches IIR country if sorted alphabetically
IEA_Countries = ['AUSTRIA','BELGIUM','CZECH','DENMARK','FINLAND','FRANCE','GERMANY','GREECE','HUNGARY','IRELAND','ITALY','NETHLAND','NORWAY','POLAND','PORTUGAL','SLOVAKIA','SPAIN','SWEDEN','SWITLAND','TURKEY','UK']
JODI_Countries = ['AT','BE','CZ','DK','FI','FR','DE','GR','HU','IE','IT','NL','NO','PL','PT','SK','ES','SE','CH','TR','GB']

folder = r'C:\Users\syedahmad\Documents\runs_model'
sub_folder = folder + "\\" + time.strftime("%Y-%m-%d")
folder_diagrams = sub_folder + "\\" + "Diagrams"
folder_scrape = folder + "\\" + "Scrape"
folder_results = folder + "\\" + time.strftime("%Y-%m-%d") + "\\" + "Results"

if os.path.isdir(sub_folder):
    shutil.rmtree(sub_folder)
else:
    pass

os.makedirs(folder_diagrams)
# os.makedirs(folder_scrape)
os.makedirs(folder_results)

address_for_error_emails = "Syed.Ahmad@petroineos.com"
sender_address_for_auto_email = "Syed.Ahmad@petroineos.com"
recipient_address_for_auto_email = "Syed.Ahmad@petroineos.com"
############################################################################


start_date = (datetime.now()-relativedelta(months=backcast_months)).strftime('%Y-%m-01')
start_date = pd.Timestamp(start_date)
end_date = start_date + relativedelta(months=months_ahead)

correlation_end_date= start_date
correlation_start_date=correlation_end_date - relativedelta(months=correlation_months_spec)

Pdate_days_before=str(Pdate_days_before+1)

#get jodi data start date which ever is earlier (1st Jan 2 years ago or 2 years 5 months from today)
graph_start_date = (datetime.now()-relativedelta(years=graph_historic_years)).strftime('%Y-01-01')
if graph_start_date<correlation_start_date.strftime('%Y-%m-01'):
    historic_start_date=graph_start_date
else:
    historic_start_date = correlation_start_date

correlation_months = len(pd.date_range(start=historic_start_date, end=correlation_end_date, freq='MS'))-1


forecast_range = pd.DataFrame({'date': pd.date_range(start=start_date, end=end_date, freq='MS')})
correlation_range = pd.DataFrame({'date': pd.date_range(start=historic_start_date, end=correlation_end_date, freq='MS')})
year_range = pd.DataFrame({'date': pd.date_range(start=datetime.now().date().replace(month=1,day=1), end=datetime.now().date().replace(month=12,day=1), freq='MS')})
timeframes = [forecast_range,correlation_range,year_range]

In [6]:
def ag_get_data(query):
    warnings.simplefilter('ignore', requests.packages.urllib3.exceptions.InsecureRequestWarning)
    params = {"Database": "OIL", "Query": query}
    resp = requests.post('https://TST-QDEV-AP1.petroineos.local:5001/genericdata/Fetch/', json=params, verify=False)
    json_result = resp.json()
    df = pd.DataFrame(json_result)
    return df

def send_error_report(email_from, emails,topic,date):
    sub = "European Refinery Runs Forecast Error" #- (" + datetime.today().strftime(format_datetime) + ")"
    body = "<font size='+1'><b>Error on "+topic+" - "+date+" .</b></font>" \
           + "<br>" \

    img = None
    email_sender = ag_email_utils.AgEmailHelper()
    email_sender.send_mail_with_embedded_images(email_from, emails, sub, body,img)


def list_to_string(locations):
    string = ""
    for location in locations:
        string += "'"+location+"', "
    string = string[:-2]
    return string

def country():
    query = ''' 
SELECT DISTINCT[country]
FROM   [refineryunit]
WHERE  country IN ( ''' + list_to_string(IIR_Countries) + ''' )
ORDER  BY country 
    '''
    df = ag_get_data(query)
    return df

In [4]:
def nameplate(j,i):
    start_date = pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%Y-%m')
    #end_date = pd.Timestamp(timeframes['date'].loc[n]).strftime('%Y-%m-%d')
    query = '''
WITH temp AS
(
         SELECT   [refineryunit].[country]                           AS country,
                  Sum([RefineryUnitCapacityMonthly].[Capacity])/1000 AS cap
         FROM     [Refinery].[RefineryUnitCapacityMonthly],
                  [refineryunit],[RefineryUnitCapacitySchedule]
         WHERE    [RefineryUnitCapacityMonthly].[Unit_ID] = [refineryunit].[unit_id]
         AND       [RefineryUnitCapacityMonthly].[Unit_ID] = [RefineryUnitCapacitySchedule].[Unit_ID]
         AND      [refineryunit].[utype_desc] = 'Atmospheric Distillation'
         AND      [refineryunit].[country] IN (''' + list_to_string(IIR_Countries) + ''')
         AND      [RefineryUnitCapacityMonthly].[DDate] LIKE ''' + "'" + start_date  + "%" + "'" + '''
         AND     NOT( ([RefineryUnit].[U_STATUS] != 'Operational' and [RefineryUnitCapacitySchedule].[EndDate]='9998-12-31'))
         GROUP BY [refineryunit].[country]) , ctry AS
(
                SELECT DISTINCT [country]
                FROM            [refineryunit]
                WHERE           country IN(''' + list_to_string(IIR_Countries) + '''))
SELECT          ctry.country,
                isnull(temp.cap,0) AS cap
FROM            ctry
LEFT OUTER JOIN temp
ON              ctry.country = temp.country
ORDER BY        ctry.country

                '''
    df0 = ag_get_data(query)
    start_date = pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%Y-%m-01')
    df0[start_date] = df0['cap']
    df0 = df0.drop(columns=['cap'])
    #df2 = country()
    #df2[start_date]= pd.Series([df1])
    #b = pd.concat([country(), df1], axis=1, join="inner")
    #df1[start_date] = df1['Planned_CAP_OFF']
    #df1 = df1.drop(columns=['Planned_CAP_OFF'])
    return print(query)


In [34]:
def p_k(j,i,n):
    start_date = pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%Y-%m-%d')
    end_date = pd.Timestamp(timeframes[j]['date'].loc[n]).strftime('%Y-%m-%d')
    query = '''
 WITH temp0 AS
(
                SELECT DISTINCT TOP (''' + Pdate_days_before +''') ([maintenancedaily].[pdate])
                FROM            [Refinery].[maintenancedaily]
                ORDER BY        [PDATE] DESC) ,temp AS
(
         SELECT   [refineryunit].[country]                                                                                                           AS country,
                  Sum([maintenancedaily].[cap_offline])/( datediff(day, ''' + "'" + start_date + "'" + ''', ''' + "'" + end_date + "'" + ''') )/1000 AS cap_off
         FROM     [Refinery].[maintenancedaily],
                  [Refinery].[refineryunit],
                  [Refinery].[refinerymaintenancebestlatest]
         WHERE    [maintenancedaily].[unit_id] = [refineryunit].[unit_id]
         AND      [maintenancedaily].[EVENT_ID] = [refinerymaintenancebestlatest].[EVENT_ID]
         AND      [refineryunit].[utype_desc] = 'Atmospheric Distillation'
         AND      [refineryunit].[country] IN (''' + list_to_string(IIR_Countries) + ''')
         AND      ((
                                    [maintenancedaily].[outage_date] >= ''' + "'" + start_date + "'" + '''
                           AND      [maintenancedaily].[outage_date] < ''' + "'" + end_date + "'" + ''' ))
         AND      [refinerymaintenancebestlatest].[event_type] = 'Planned'
         AND      (
                           [refinerymaintenancebestlatest].[DERATE] IS NULL
                  OR       ((
                                             [refinerymaintenancebestlatest].[E_CAUSE] = 'Electrical'
                                    OR       [refinerymaintenancebestlatest].[E_CAUSE] = 'Mechanical'
                                    OR       [refinerymaintenancebestlatest].[E_CAUSE] = 'Weather'
                                    OR       [refinerymaintenancebestlatest].[E_CAUSE] IS NULL)
                           AND      [refinerymaintenancebestlatest].[DERATE] = 'DERATE'))
         AND      [maintenancedaily].[pdate] =
                  (
                           SELECT TOP(1)
                                    [pdate]
                           FROM     temp0
                           ORDER BY [PDATE])
         GROUP BY [refineryunit].[country]) , ctry AS
(
                SELECT DISTINCT [country]
                FROM            [Refinery].[refineryunit]
                WHERE           country IN(''' + list_to_string(IIR_Countries) + '''))
SELECT          ctry.country           AS country,
                isnull(temp.cap_off,0) AS planned_cap_off
FROM            ctry
LEFT OUTER JOIN temp
ON              ctry.country = temp.country
ORDER BY        ctry.country
                '''

    @timer(timeout_sec)
    def dff1():
        df1 = ag_get_data(query)
        # df2 = country()
        # df2[start_date]= pd.Series([df1])
        # b = pd.concat([country(), df1], axis=1, join="inner")
        df1[start_date] = df1['planned_cap_off']
        df1 = df1.drop(columns=['planned_cap_off'])
        return df1

    try:
        df1=dff1()
    except multiprocessing.context.TimeoutError:
        try:
            df1 = dff1()
        except multiprocessing.context.TimeoutError:
            print('ERROR ON '+start_date)
            send_error_report(address_for_error_emails, address_for_error_emails,'planned known TAR',start_date)
            if os.path.isdir(sub_folder):
                shutil.rmtree(sub_folder)
            else:  ## Show an error ##
                print("Error: %s file not found" % sub_folder)
            df1 = country()
            df1[start_date] = 'Error'
            df1.replace(np.nan, 0, inplace=True)
    return print(query)

In [35]:
for i in zip(range(0,months_ahead)):
     p_k(0,i)

TypeError: p_k() missing 1 required positional argument: 'n'

In [8]:
months_ahead

24

In [23]:
timeframes

[         date
 0  2022-01-01
 1  2022-02-01
 2  2022-03-01
 3  2022-04-01
 4  2022-05-01
 5  2022-06-01
 6  2022-07-01
 7  2022-08-01
 8  2022-09-01
 9  2022-10-01
 10 2022-11-01
 11 2022-12-01
 12 2023-01-01
 13 2023-02-01
 14 2023-03-01
 15 2023-04-01
 16 2023-05-01
 17 2023-06-01
 18 2023-07-01
 19 2023-08-01
 20 2023-09-01
 21 2023-10-01
 22 2023-11-01
 23 2023-12-01
 24 2024-01-01,
          date
 0  2017-01-01
 1  2017-02-01
 2  2017-03-01
 3  2017-04-01
 4  2017-05-01
 5  2017-06-01
 6  2017-07-01
 7  2017-08-01
 8  2017-09-01
 9  2017-10-01
 10 2017-11-01
 11 2017-12-01
 12 2018-01-01
 13 2018-02-01
 14 2018-03-01
 15 2018-04-01
 16 2018-05-01
 17 2018-06-01
 18 2018-07-01
 19 2018-08-01
 20 2018-09-01
 21 2018-10-01
 22 2018-11-01
 23 2018-12-01
 24 2019-01-01
 25 2019-02-01
 26 2019-03-01
 27 2019-04-01
 28 2019-05-01
 29 2019-06-01
 30 2019-07-01
 31 2019-08-01
 32 2019-09-01
 33 2019-10-01
 34 2019-11-01
 35 2019-12-01
 36 2020-01-01
 37 2020-02-01
 38 2020-03-01
 39 2020-

In [33]:
timeframes[0]['date'].loc[3]

Timestamp('2022-04-01 00:00:00')

In [26]:
for i in zip(range(0,months_ahead)):
    print(i)

(0,)
(1,)
(2,)
(3,)
(4,)
(5,)
(6,)
(7,)
(8,)
(9,)
(10,)
(11,)
(12,)
(13,)
(14,)
(15,)
(16,)
(17,)
(18,)
(19,)
(20,)
(21,)
(22,)
(23,)


In [21]:
for i in zip(range(0,months_ahead)):    
    start_date = pd.Timestamp(timeframes[0]['date'].loc[i]).strftime('%Y-%m')
    print(start_date)

2022-01
2022-02
2022-03
2022-04
2022-05
2022-06
2022-07
2022-08
2022-09
2022-10
2022-11
2022-12
2023-01
2023-02
2023-03
2023-04
2023-05
2023-06
2023-07
2023-08
2023-09
2023-10
2023-11
2023-12
